# 🌎 Welcome to the CSE151B Spring 2025 Climate Emulation Competition!

Thank you for participating in this exciting challenge focused on building machine learning models to emulate complex climate systems.  
This notebook is provided as a **starter template** to help you:

- Understand how to load and preprocess the dataset  
- Construct a baseline model  
- Train and evaluate predictions using a PyTorch Lightning pipeline  
- Format your predictions for submission to the leaderboard  

You're encouraged to:
- Build on this structure or replace it entirely
- Try more advanced models and training strategies
- Incorporate your own ideas to push the boundaries of what's possible

If you're interested in developing within a repository structure and/or use helpful tools like configuration management (based on Hydra) and logging (with Weights & Biases), we recommend checking out the following Github repo. Such a structure can be useful when running multiple experiments and trying various research ideas.

👉 [https://github.com/salvaRC/cse151b-spring2025-competition](https://github.com/salvaRC/cse151b-spring2025-competition)

Good luck, have fun, and we hope you learn a lot through this process!


### 📦 Install Required Libraries
We install the necessary Python packages for data loading, deep learning, and visualization.


In [5]:
!pip install xarray zarr dask lightning matplotlib wandb cftime einops --quiet

import os
from datetime import datetime
import numpy as np
import xarray as xr
import dask.array as da
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import lightning.pytorch as pl


### ⚙️ Configuration Setup  
Define all model, data, and training hyperparameters in one place for easy control and reproducibility.

### 📊 Data Configuration

We define the dataset settings used for training and evaluation. This includes:

- **`path`**: Path to the `.zarr` dataset containing monthly climate variables from CMIP6 simulations.
- **`input_vars`**: Climate forcing variables (e.g., CO₂, CH₄) used as model inputs.
- **`output_vars`**: Target variables to predict — surface air temperature (`tas`) and precipitation (`pr`).
- **`target_member_id`**: Ensemble member to use from the simulations (each SSP has 3) for target variables.
- **`train_ssps`**: SSP scenarios used for training (low to high emissions).
- **`test_ssp`**: Scenario held out for evaluation (Must be set to SSP245).
- **`test_months`**: Number of months to include in the test split (Must be set to 120).
- **`batch_size`** and **`num_workers`**: Data loading parameters for PyTorch training.

These settings reflect how the challenge is structured: models must learn from some emission scenarios and generalize to unseen ones.

> ⚠️ **Important:** Do **not modify** the following test settings:
>
> - `test_ssp` must remain **`ssp245`**, which is the held-out evaluation scenario.
> - `test_months` must be **`120`**, corresponding to the last 10 years (monthly resolution) of the scenario.



In [17]:
#NOTE Change the data directory according to where you have your zarr files stored
config = {
    "data": {
        "path": "/home/wcw003/private/processed_data_cse151b_v2_corrupted_ssp245.zarr",
        "input_vars": ["CO2", "SO2", "CH4", "BC", "rsdt"],
        "output_vars": ["tas", "pr"],
        "target_member_id": 0,
        "train_ssps": ["ssp126", "ssp370", "ssp585"],
        "test_ssp": "ssp245",
        "test_months": 360,
        "batch_size": 8,
        "num_workers": 4,
    },
    "model": {
        "type": "simple_cnn",
        "kernel_size": 3,
        "init_dim": 64,
        "depth": 4,
        "dropout_rate": 0.1,
    },
    "training": {
        "lr": 1e-3,
    },
    "trainer": {
        "max_epochs": 30, # originally 10
        "accelerator": "auto",
        "devices": "auto",
        "precision": 32,
        "deterministic": True,
        "num_sanity_val_steps": 0
    },
    "seed": 42,
}
pl.seed_everything(config["seed"])  # Set seed for reproducibility

Seed set to 42


42

### 🔧 Spatial Weighting Utility Function

This cell sets up utility functions for reproducibility and spatial weighting:

- **`get_lat_weights(latitude_values)`**: Computes cosine-based area weights for each latitude, accounting for the Earth's curvature. This is critical for evaluating global climate metrics fairly — grid cells near the equator represent larger surface areas than those near the poles.


In [18]:
def get_lat_weights(latitude_values):
    lat_rad = np.deg2rad(latitude_values)
    weights = np.cos(lat_rad)
    return weights / np.mean(weights)

### 🧠 SimpleCNN: A Residual Convolutional Baseline

This is a lightweight baseline model designed to capture spatial patterns in global climate data using convolutional layers.

- The architecture starts with a **convolution + batch norm + ReLU** block to process the input channels.
- It then applies a series of **residual blocks** to extract increasingly abstract spatial features. These help preserve gradient flow during training.
- Finally, a few convolutional layers reduce the feature maps down to the desired number of output channels (`tas` and `pr`).

This model only serves as a **simple baseline for climate emulation**. 

We encourage you to build and experiment with your own models and ideas.


In [19]:
#first modify
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super().__init__()
        # Increase kernel size for capturing larger spatial patterns in climate data
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=5, stride=stride, padding=2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        # Use LeakyReLU for better gradient flow with climate data
        self.act = nn.LeakyReLU(0.2, inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.skip(identity)
        return self.act(out)

class SimpleCNN(nn.Module):
    def __init__(self, n_input_channels, n_output_channels, kernel_size=5, init_dim=128, depth=6, dropout_rate=0.3):
        super().__init__()
        # Increased initial dimensions and depth for climate modeling capacity
        self.initial = nn.Sequential(
            nn.Conv2d(n_input_channels, init_dim, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.BatchNorm2d(init_dim),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        self.res_blocks = nn.ModuleList()
        current_dim = init_dim
        for i in range(depth):
            out_dim = current_dim * 2 if i < depth - 1 else current_dim
            self.res_blocks.append(ResidualBlock(current_dim, out_dim))
            if i < depth - 1:
                current_dim *= 2
                
        # Added spatial attention for focusing on important climate patterns
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(current_dim, 1, kernel_size=1),
            nn.Sigmoid()
        )
        
        self.dropout = nn.Dropout2d(dropout_rate)
        
        # Modified final layers for better climate variable prediction
        self.final = nn.Sequential(
            nn.Conv2d(current_dim, current_dim // 2, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.BatchNorm2d(current_dim // 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(current_dim // 2, current_dim // 4, kernel_size=3, padding=1),
            nn.BatchNorm2d(current_dim // 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(current_dim // 4, n_output_channels, kernel_size=1),
        )

    def forward(self, x):
        x = self.initial(x)
        for res_block in self.res_blocks:
            x = res_block(x)
            
        # Apply spatial attention
        attention = self.spatial_attention(x)
        x = x * attention
        
        return self.final(self.dropout(x))


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np
from torch.utils.data import DataLoader

class TimeDistributed(nn.Module):
    """Applies a module over multiple time steps"""
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        # Reshape to (batch_size * time_steps, ...)
        batch_size, time_steps = x.size(0), x.size(1)
        x_reshaped = x.contiguous().view(batch_size * time_steps, *x.size()[2:])
        
        # Apply module
        y = self.module(x_reshaped)
        
        # Reshape back
        return y.contiguous().view(batch_size, time_steps, *y.size()[1:])

class SpatialAttention(nn.Module):
    """Spatial attention mechanism for focusing on important regions"""
    def __init__(self, in_channels):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=1)
        
    def forward(self, x):
        # Generate attention map
        attention = torch.sigmoid(self.conv(x))
        return x * attention

class TemporalAttention(nn.Module):
    """Temporal attention mechanism for focusing on important time steps"""
    def __init__(self, hidden_dim):
        super(TemporalAttention, self).__init__()
        self.fc = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        # x shape: [batch, time_steps, features]
        # Generate attention weights
        attention_weights = F.softmax(self.fc(x), dim=1)
        # Apply attention
        context = torch.sum(x * attention_weights, dim=1)
        return context

class ResidualBlock(nn.Module):
    """Residual block with larger kernels for climate patterns"""
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, 
                              stride=stride, padding=kernel_size//2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size,
                              stride=1, padding=kernel_size//2, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Shortcut connection if dimensions change
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class TimeSeriesClimateCNN(nn.Module):
    """CNN model for climate time series prediction with optional LSTM"""
    def __init__(self, n_input_channels, n_output_channels, seq_length=12, 
                 kernel_size=5, init_dim=64, depth=4, dropout_rate=0.3, use_lstm=True):
        super(TimeSeriesClimateCNN, self).__init__()
        
        self.n_input_channels = n_input_channels
        self.n_output_channels = n_output_channels
        self.seq_length = seq_length
        self.use_lstm = use_lstm
        
        # Initial convolutional layer
        self.initial_conv = TimeDistributed(
            nn.Conv2d(n_input_channels, init_dim, kernel_size=kernel_size, 
                     stride=1, padding=kernel_size//2)
        )
        
        # Residual blocks with increasing channels
        self.res_blocks = nn.ModuleList()
        channels = [init_dim * (2**i) for i in range(depth)]
        
        for i in range(depth-1):
            block = TimeDistributed(
                ResidualBlock(channels[i], channels[i+1], kernel_size=kernel_size, stride=2)
            )
            self.res_blocks.append(block)
        
        # Spatial attention after CNN layers
        self.spatial_attention = TimeDistributed(SpatialAttention(channels[-1]))
        
        # Global average pooling to reduce spatial dimensions
        self.gap = TimeDistributed(nn.AdaptiveAvgPool2d(1))
        
        # LSTM for temporal modeling
        if use_lstm:
            self.lstm = nn.LSTM(
                input_size=channels[-1],
                hidden_size=channels[-1],
                num_layers=2,
                batch_first=True,
                dropout=dropout_rate,
                bidirectional=True
            )
            lstm_output_dim = channels[-1] * 2  # bidirectional
        else:
            lstm_output_dim = channels[-1]
        
        # Temporal attention
        self.temporal_attention = TemporalAttention(lstm_output_dim)
        
        # Final prediction layers
        self.dropout = nn.Dropout(dropout_rate)
        
        # Upsampling to original resolution
        self.upsampling = nn.Sequential(
            nn.ConvTranspose2d(lstm_output_dim, channels[-2], kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(channels[-2], channels[-3], kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(channels[-3], n_output_channels, kernel_size=4, stride=2, padding=1)
        )
        
    def forward(self, x):
        # x shape: [batch, time_steps, channels, height, width]
        batch_size = x.size(0)
        
        # Apply initial convolution
        x = self.initial_conv(x)
        
        # Apply residual blocks
        for block in self.res_blocks:
            x = block(x)
        
        # Apply spatial attention
        x = self.spatial_attention(x)
        
        # Global average pooling
        x = self.gap(x)
        x = x.view(batch_size, self.seq_length, -1)  # [batch, time_steps, channels]
        
        # Apply LSTM if enabled
        if self.use_lstm:
            x, _ = self.lstm(x)
        
        # Apply temporal attention to focus on important time steps
        x = self.temporal_attention(x)  # [batch, channels]
        
        # Reshape for upsampling
        x = x.view(batch_size, -1, 1, 1)
        
        # Apply dropout
        x = self.dropout(x)
        
        # Upsampling to get final prediction
        x = self.upsampling(x)
        
        return x

class ClimateTransformerCNN(nn.Module):
    """CNN model with transformer for temporal modeling"""
    def __init__(self, n_input_channels, n_output_channels, seq_length=12, 
                 kernel_size=5, init_dim=64, depth=4, n_heads=8, dropout_rate=0.3):
        super(ClimateTransformerCNN, self).__init__()
        
        self.n_input_channels = n_input_channels
        self.n_output_channels = n_output_channels
        self.seq_length = seq_length
        
        # Initial convolutional layer
        self.initial_conv = TimeDistributed(
            nn.Conv2d(n_input_channels, init_dim, kernel_size=kernel_size, 
                     stride=1, padding=kernel_size//2)
        )
        
        # Residual blocks with increasing channels
        self.res_blocks = nn.ModuleList()
        channels = [init_dim * (2**i) for i in range(depth)]
        
        for i in range(depth-1):
            block = TimeDistributed(
                ResidualBlock(channels[i], channels[i+1], kernel_size=kernel_size, stride=2)
            )
            self.res_blocks.append(block)
        
        # Spatial attention after CNN layers
        self.spatial_attention = TimeDistributed(SpatialAttention(channels[-1]))
        
        # Global average pooling to reduce spatial dimensions
        self.gap = TimeDistributed(nn.AdaptiveAvgPool2d(1))
        
        # Transformer encoder for temporal modeling
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=channels[-1],
            nhead=n_heads,
            dim_feedforward=channels[-1]*4,
            dropout=dropout_rate,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
        # Temporal attention
        self.temporal_attention = TemporalAttention(channels[-1])
        
        # Final prediction layers
        self.dropout = nn.Dropout(dropout_rate)
        
        # Upsampling to original resolution
        self.upsampling = nn.Sequential(
            nn.ConvTranspose2d(channels[-1], channels[-2], kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(channels[-2], channels[-3], kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(channels[-3], n_output_channels, kernel_size=4, stride=2, padding=1)
        )
        
    def forward(self, x):
        # x shape: [batch, time_steps, channels, height, width]
        batch_size = x.size(0)
        
        # Apply initial convolution
        x = self.initial_conv(x)
        
        # Apply residual blocks
        for block in self.res_blocks:
            x = block(x)
        
        # Apply spatial attention
        x = self.spatial_attention(x)
        
        # Global average pooling
        x = self.gap(x)
        x = x.view(batch_size, self.seq_length, -1)  # [batch, time_steps, channels]
        
        # Apply transformer encoder
        x = self.transformer_encoder(x)
        
        # Apply temporal attention to focus on important time steps
        x = self.temporal_attention(x)  # [batch, channels]
        
        # Reshape for upsampling
        x = x.view(batch_size, -1, 1, 1)
        
        # Apply dropout
        x = self.dropout(x)
        
        # Upsampling to get final prediction
        x = self.upsampling(x)
        
        return x


In [21]:
#original
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding=kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride), nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.skip(identity)
        return self.relu(out)

class SimpleCNN(nn.Module):
    def __init__(self, n_input_channels, n_output_channels, kernel_size=3, init_dim=64, depth=4, dropout_rate=0.2):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(n_input_channels, init_dim, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.BatchNorm2d(init_dim),
            nn.ReLU(inplace=True),
        )
        
        self.res_blocks = nn.ModuleList()
        current_dim = init_dim
        for i in range(depth):
            out_dim = current_dim * 2 if i < depth - 1 else current_dim
            self.res_blocks.append(ResidualBlock(current_dim, out_dim))
            if i < depth - 1:
                current_dim *= 2
        
        self.dropout = nn.Dropout2d(dropout_rate)
        self.final = nn.Sequential(
            nn.Conv2d(current_dim, current_dim // 2, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.BatchNorm2d(current_dim // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(current_dim // 2, n_output_channels, kernel_size=1),
        )

    def forward(self, x):
        x = self.initial(x)
        for res_block in self.res_blocks:
            x = res_block(x)
        return self.final(self.dropout(x))


### 📐 Normalizer: Z-Score Scaling for Climate Inputs & Outputs

This class handles **Z-score normalization**, a crucial preprocessing step for stable and efficient neural network training:

- **`set_input_statistics(mean, std)` / `set_output_statistics(...)`**: Store the mean and standard deviation computed from the training data for later use.
- **`normalize(data, data_type)`**: Standardizes the data using `(x - mean) / std`. This is applied separately to inputs and outputs.
- **`inverse_transform_output(data)`**: Converts model predictions back to the original physical units (e.g., Kelvin for temperature, mm/day for precipitation).

Normalizing the data ensures the model sees inputs with similar dynamic ranges and avoids biases caused by different variable scales.


In [22]:
class Normalizer:
    def __init__(self):
        self.mean_in, self.std_in = None, None
        self.mean_out, self.std_out = None, None

    def set_input_statistics(self, mean, std):
        self.mean_in = mean
        self.std_in = std

    def set_output_statistics(self, mean, std):
        self.mean_out = mean
        self.std_out = std

    def normalize(self, data, data_type):
        if data_type == "input":
            return (data - self.mean_in) / self.std_in
        elif data_type == "output":
            return (data - self.mean_out) / self.std_out

    def inverse_transform_output(self, data):
        return data * self.std_out + self.mean_out


### 🌍 Data Module: Loading, Normalization, and Splitting

This section handles the entire data pipeline, from loading the `.zarr` dataset to preparing PyTorch-ready DataLoaders.

#### `ClimateDataset`
- A simple PyTorch `Dataset` wrapper that preloads the entire (normalized) dataset into memory using Dask.
- Converts the data to PyTorch tensors and handles any `NaN` checks up front.

#### `ClimateDataModule`
A PyTorch Lightning `DataModule` that handles:
- ✅ **Loading data** from different SSP scenarios and ensemble members
- ✅ **Broadcasting non-spatial inputs** (like CO₂) to match spatial grid size
- ✅ **Normalization** using mean/std computed from training data only
- ✅ **Splitting** into training, validation, and test sets:
  - Training: All months from selected SSPs (except last 10 years of SSP370)
  - Validation: Last 10 years (120 months) of SSP370
  - Test: Last 10 years of SSP245 (unseen scenario)
- ✅ **Batching** and parallelized data loading via PyTorch `DataLoader`s
- ✅ **Latitude-based area weighting** for fair climate metric evaluation
- Shape of the inputs are Batch_Size X 5 (num_input_variables) X 48 X 72
- Shape of ouputputs are Batch_Size X 2 (num_output_variables) X 48 X 72

> ℹ️ **Note:** You likely won’t need to modify this class but feel free to make modifications if you want to inlcude different ensemble members to feed more data to your models


In [23]:
class ClimateDataset(Dataset):
    def __init__(self, inputs_dask, outputs_dask, output_is_normalized=True):
        self.size = inputs_dask.shape[0]
        print(f"Creating dataset with {self.size} samples...")

        inputs_np = inputs_dask.compute()
        outputs_np = outputs_dask.compute()

        self.inputs = torch.from_numpy(inputs_np).float()
        self.outputs = torch.from_numpy(outputs_np).float()

        if torch.isnan(self.inputs).any() or torch.isnan(self.outputs).any():
            raise ValueError("NaNs found in dataset")

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]


class ClimateDataModule(pl.LightningDataModule):
    def __init__(
        self,
        path,
        input_vars,
        output_vars,
        train_ssps,
        test_ssp,
        target_member_id,
        val_split=0.1,
        test_months=360,
        batch_size=32,
        num_workers=0,
        seed=42,
    ):
        super().__init__()
        self.path = path
        self.input_vars = input_vars
        self.output_vars = output_vars
        self.train_ssps = train_ssps
        self.test_ssp = test_ssp
        self.target_member_id = target_member_id
        self.val_split = val_split
        self.test_months = test_months
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.seed = seed
        self.normalizer = Normalizer()

    def prepare_data(self):
        assert os.path.exists(self.path), f"Data path not found: {self.path}"

    def setup(self, stage=None):
        ds = xr.open_zarr(self.path, consolidated=False, chunks={"time": 24})
        spatial_template = ds["rsdt"].isel(time=0, ssp=0, drop=True)

        def load_ssp(ssp):
            input_dask, output_dask = [], []
            for var in self.input_vars:
                da_var = ds[var].sel(ssp=ssp)
                if "latitude" in da_var.dims:
                    da_var = da_var.rename({"latitude": "y", "longitude": "x"})
                if "member_id" in da_var.dims:
                    da_var = da_var.sel(member_id=self.target_member_id)
                if set(da_var.dims) == {"time"}:
                    da_var = da_var.broadcast_like(spatial_template).transpose("time", "y", "x")
                input_dask.append(da_var.data)

            for var in self.output_vars:
                da_out = ds[var].sel(ssp=ssp, member_id=self.target_member_id)
                if "latitude" in da_out.dims:
                    da_out = da_out.rename({"latitude": "y", "longitude": "x"})
                output_dask.append(da_out.data)

            return da.stack(input_dask, axis=1), da.stack(output_dask, axis=1)

        train_input, train_output, val_input, val_output = [], [], None, None

        for ssp in self.train_ssps:
            x, y = load_ssp(ssp)
            if ssp == "ssp370":
                val_input = x[-self.test_months:]
                val_output = y[-self.test_months:]
                train_input.append(x[:-self.test_months])
                train_output.append(y[:-self.test_months])
            else:
                train_input.append(x)
                train_output.append(y)

        train_input = da.concatenate(train_input, axis=0)
        train_output = da.concatenate(train_output, axis=0)

        self.normalizer.set_input_statistics(
            mean=da.nanmean(train_input, axis=(0, 2, 3), keepdims=True).compute(),
            std=da.nanstd(train_input, axis=(0, 2, 3), keepdims=True).compute(),
        )
        self.normalizer.set_output_statistics(
            mean=da.nanmean(train_output, axis=(0, 2, 3), keepdims=True).compute(),
            std=da.nanstd(train_output, axis=(0, 2, 3), keepdims=True).compute(),
        )

        train_input_norm = self.normalizer.normalize(train_input, "input")
        train_output_norm = self.normalizer.normalize(train_output, "output")
        val_input_norm = self.normalizer.normalize(val_input, "input")
        val_output_norm = self.normalizer.normalize(val_output, "output")

        test_input, test_output = load_ssp(self.test_ssp)
        test_input = test_input[-self.test_months:]
        test_output = test_output[-self.test_months:]
        test_input_norm = self.normalizer.normalize(test_input, "input")

        self.train_dataset = ClimateDataset(train_input_norm, train_output_norm)
        self.val_dataset = ClimateDataset(val_input_norm, val_output_norm)
        self.test_dataset = ClimateDataset(test_input_norm, test_output, output_is_normalized=False)

        self.lat = spatial_template.y.values
        self.lon = spatial_template.x.values
        self.area_weights = xr.DataArray(get_lat_weights(self.lat), dims=["y"], coords={"y": self.lat})

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True,
                          num_workers=self.num_workers, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def get_lat_weights(self):
        return self.area_weights

    def get_coords(self):
        return self.lat, self.lon

### ⚡ ClimateEmulationModule: Lightning Wrapper for Climate Model Emulation

This is the core model wrapper built with **PyTorch Lightning**, which organizes the training, validation, and testing logic for the climate emulation task. Lightning abstracts away much of the boilerplate code in PyTorch-based deep learning workflows, making it easier to scale models.

#### ✅ Key Features

- **`training_step` / `validation_step` / `test_step`**: Standard Lightning hooks for computing loss and predictions at each stage. The loss used is **Mean Squared Error (MSE)**.

- **Normalization-aware outputs**:
  - During validation and testing, predictions and targets are denormalized before evaluation using stored mean/std statistics.
  - This ensures evaluation is done in real-world units (Kelvin and mm/day).

- **Metric Evaluation** via `_evaluate()`:
  For each variable (`tas`, `pr`), it calculates:
  - **Monthly Area-Weighted RMSE**
  - **Time-Mean RMSE** (RMSE on 10-year average's)
  - **Time-Stddev MAE** (MAE on 10-year standard deviation; a measure of temporal variability)
    
  These metrics reflect the competition's evaluation criteria and are logged and printed.

- **Kaggle Submission Writer**:
  After testing, predictions are saved to a `.csv` file in the required Kaggle format via `_save_submission()`.

- **Saving Predictions for Visualization**:
  - Validation predictions are saved tao `val_preds.npy` and `val_trues.npy`
  - These can be loaded later for visual inspection of the model's performance.

 🔧 **Feel free to modify any part of this module** (loss functions, evaluation, training logic) to better suit your model or training pipeline / Use pure PyTorch etc.

⚠️ The **final submission `.csv` file must strictly follow the format and naming convention used in `_save_submission()`**, as these `ID`s are used to match predictions to the hidden test set during evaluation.



In [24]:
import pandas as pd

class ClimateEmulationModule(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-4):
        super().__init__()
        self.model = model
        self.save_hyperparameters(ignore=['model']) # Save all hyperparameters except the model to self.hparams.<param_name>
        self.criterion = nn.MSELoss()
        self.normalizer = None
        self.val_preds, self.val_targets = [], []
        self.test_preds, self.test_targets = [], []

    def forward(self, x):
        return self.model(x)

    def on_fit_start(self):
        self.normalizer = self.trainer.datamodule.normalizer  # Get the normalizer from the datamodule (see above)

    def training_step(self, batch, batch_idx):
        x, y = batch # Unpack inputs and targets (this is the output of the _getitem_ method in the Dataset above)
        y_hat = self(x)   # Forward pass
        loss = self.criterion(y_hat, y)  # Calculate loss
        self.log("train/loss", loss)  # Log loss for tracking
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val/loss", loss)

        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = self.normalizer.inverse_transform_output(y.detach().cpu().numpy())
        self.val_preds.append(y_hat_np)
        self.val_targets.append(y_np)

        return loss

    def on_validation_epoch_end(self):
        # Concatenate all predictions and ground truths from each val step/batch into one array
        preds = np.concatenate(self.val_preds, axis=0)
        trues = np.concatenate(self.val_targets, axis=0)
        self._evaluate(preds, trues, phase="val")
        np.save("val_preds.npy", preds)
        np.save("val_trues.npy", trues)
        self.val_preds.clear()
        self.val_targets.clear()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = y.detach().cpu().numpy()
        self.test_preds.append(y_hat_np)
        self.test_targets.append(y_np)

    def on_test_epoch_end(self):
        # Concatenate all predictions and ground truths from each test step/batch into one array
        preds = np.concatenate(self.test_preds, axis=0)
        trues = np.concatenate(self.test_targets, axis=0)
        self._evaluate(preds, trues, phase="test")
        self._save_submission(preds)
        self.test_preds.clear()
        self.test_targets.clear()

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def _evaluate(self, preds, trues, phase="val"):
        datamodule = self.trainer.datamodule
        area_weights = datamodule.get_lat_weights()
        lat, lon = datamodule.get_coords()
        time = np.arange(preds.shape[0])
        output_vars = datamodule.output_vars

        for i, var in enumerate(output_vars):
            p = preds[:, i]
            t = trues[:, i]
            p_xr = xr.DataArray(p, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})
            t_xr = xr.DataArray(t, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})

            # RMSE
            rmse = np.sqrt(((p_xr - t_xr) ** 2).weighted(area_weights).mean(("time", "y", "x")).item())
            # RMSE of time-mean
            mean_rmse = np.sqrt(((p_xr.mean("time") - t_xr.mean("time")) ** 2).weighted(area_weights).mean(("y", "x")).item())
            # MAE of time-stddev
            std_mae = np.abs(p_xr.std("time") - t_xr.std("time")).weighted(area_weights).mean(("y", "x")).item()

            print(f"[{phase.upper()}] {var}: RMSE={rmse:.4f}, Time-Mean RMSE={mean_rmse:.4f}, Time-Stddev MAE={std_mae:.4f}")
            self.log_dict({
                f"{phase}/{var}/rmse": rmse,
                f"{phase}/{var}/time_mean_rmse": mean_rmse,
                f"{phase}/{var}/time_std_mae": std_mae,
            })

    def _save_submission(self, predictions):
        datamodule = self.trainer.datamodule
        lat, lon = datamodule.get_coords()
        output_vars = datamodule.output_vars
        time = np.arange(predictions.shape[0])

        rows = []
        for t_idx, t in enumerate(time):
            for var_idx, var in enumerate(output_vars):
                for y_idx, y in enumerate(lat):
                    for x_idx, x in enumerate(lon):
                        row_id = f"t{t_idx:03d}_{var}_{y:.2f}_{x:.2f}"
                        pred = predictions[t_idx, var_idx, y_idx, x_idx]
                        rows.append({"ID": row_id, "Prediction": pred})

        df = pd.DataFrame(rows)
        os.makedirs("submissions", exist_ok=True)
        filepath = f"submissions/kaggle_submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filepath, index=False)
        print(f"✅ Submission saved to: {filepath}")

### ⚡ Training & Evaluation with PyTorch Lightning

This block sets up and runs the training and testing pipeline using **PyTorch Lightning’s `Trainer`**, which abstracts away much of the boilerplate in deep learning workflows.

- **Modular Setup**:
  - `datamodule`: Handles loading, normalization, and batching of climate data.
  - `model`: A convolutional neural network that maps climate forcings to predicted outputs.
  - `lightning_module`: Wraps the model with training/validation/test logic and metric evaluation.

- **Trainer Flexibility**:
  The `Trainer` accepts a wide range of configuration options from `config["trainer"]`, including:
  - Number of epochs
  - Precision (e.g., 16-bit or 32-bit)
  - Device configuration (CPU, GPU, or TPU)
  - Determinism, logging, callbacks, and more

In [25]:
from models_3 import SequentialClimateCNN

In [27]:
from pytorch_lightning.loggers import CSVLogger
import pandas as pd

csv_logger = CSVLogger(save_dir="logs/", name="seqcnn_30")

In [ ]:
# from models import SequentialClimateCNN

datamodule = ClimateDataModule(**config["data"])
model = SequentialClimateCNN(
    n_input_channels=len(config["data"]["input_vars"]),
    n_output_channels=len(config["data"]["output_vars"]),
    seq_length=config["data"].get("seq_length", 12),
    kernel_size=config["model"].get("kernel_size", 5),
    hidden_dim=config["model"].get("init_dim", 128),
    spatial_depth=config["model"].get("depth", 5),
    dropout_rate=config["model"].get("dropout_rate", 0.3)
)
lightning_module = ClimateEmulationModule(model, learning_rate=config["training"]["lr"])

trainer = pl.Trainer(**config["trainer"], logger=csv_logger)
trainer.fit(lightning_module, datamodule=datamodule) 

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/wcw003/.local/lib/python3.11/site-packages/zarr/core/group.py:3301: UserWarning: Object at .DS_Store is not recognized as a component of a Zarr hierarchy.
  warnings.warn(


Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                 | Params | Mode 
-----------------------------------------------------------
0 | model     | SequentialClimateCNN | 38.7 M | train
1 | criterion | MSELoss              | 0      | train
-----------------------------------------------------------
38.7 M    Trainable params
0         Non-trainable params
38.7 M    Total params
154.911   Total estimated model params size (MB)
100       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
from models_2 import ClimateTransformerCNN

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TimeDistributed(nn.Module):
    """Applies a module over multiple time steps"""
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        # Reshape to (batch_size * time_steps, ...)
        batch_size, time_steps = x.size(0), x.size(1)
        x_reshaped = x.contiguous().view(batch_size * time_steps, *x.size()[2:])
        
        # Apply module
        y = self.module(x_reshaped)
        
        # Reshape back
        return y.contiguous().view(batch_size, time_steps, *y.size()[1:])

class SpatialAttention(nn.Module):
    """Spatial attention mechanism for focusing on important regions"""
    def __init__(self, in_channels):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=1)
        
    def forward(self, x):
        # Generate attention map
        attention = torch.sigmoid(self.conv(x))
        return x * attention

class TemporalAttention(nn.Module):
    """Temporal attention mechanism for focusing on important time steps"""
    def __init__(self, hidden_dim):
        super(TemporalAttention, self).__init__()
        self.fc = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        # x shape: [batch, time_steps, features]
        # Generate attention weights
        attention_weights = F.softmax(self.fc(x), dim=1)
        # Apply attention
        context = torch.sum(x * attention_weights, dim=1)
        return context

class ResidualBlock(nn.Module):
    """Residual block with larger kernels for climate patterns"""
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, 
                              stride=stride, padding=kernel_size//2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.act = nn.LeakyReLU(0.2, inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, 
                              padding=kernel_size//2)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Skip connection
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.skip(identity)
        return self.act(out)

class TimeSeriesClimateCNN(nn.Module):
    """
    Time Series CNN for climate emulation
    Combines temporal processing with spatial CNN features
    """
    def __init__(
        self, 
        n_input_channels, 
        n_output_channels, 
        seq_length=12,
        kernel_size=5, 
        init_dim=64, 
        depth=4, 
        dropout_rate=0.3,
        use_lstm=True
    ):
        super(TimeSeriesClimateCNN, self).__init__()
        
        self.seq_length = seq_length
        self.use_lstm = use_lstm
        
        # Initial convolution applied to each time step
        self.initial_conv = nn.Sequential(
            nn.Conv2d(n_input_channels, init_dim, kernel_size=kernel_size, 
                     padding=kernel_size//2),
            nn.BatchNorm2d(init_dim),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        # Time-distributed CNN blocks
        self.res_blocks = nn.ModuleList()
        current_dim = init_dim
        
        for i in range(depth):
            out_dim = current_dim * 2 if i < depth - 1 else current_dim
            self.res_blocks.append(
                TimeDistributed(ResidualBlock(current_dim, out_dim))
            )
            if i < depth - 1:
                current_dim *= 2
        
        # Spatial attention after CNN processing
        self.spatial_attention = TimeDistributed(SpatialAttention(current_dim))
        
        # Global average pooling to reduce spatial dimensions
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        
        # LSTM for temporal processing
        if use_lstm:
            self.lstm = nn.LSTM(
                input_size=current_dim,
                hidden_size=current_dim*2,
                num_layers=2,
                batch_first=True,
                dropout=dropout_rate if depth > 1 else 0
            )
            lstm_output_dim = current_dim*2
        else:
            lstm_output_dim = current_dim
            
        # Temporal attention
        self.temporal_attention = TemporalAttention(lstm_output_dim)
        
        # Final prediction layers
        self.decoder = nn.Sequential(
            nn.Conv2d(lstm_output_dim, current_dim, kernel_size=3, padding=1),
            nn.BatchNorm2d(current_dim),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(dropout_rate),
            nn.Conv2d(current_dim, current_dim//2, kernel_size=3, padding=1),
            nn.BatchNorm2d(current_dim//2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(current_dim//2, n_output_channels, kernel_size=1)
        )

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        
        # Instead of trying to reshape the input based on sequence length,
        # we'll adjust our approach to handle the actual input format
        
        # The input has 5 channels, which suggests it's a single time step with 5 features
        # Let's replicate this input across the sequence length
        
        # Remove the debug print in production code
        # print(f"Input shape: {x.shape}, Total elements: {x.numel()}")
        
        # Apply initial convolution directly
        x = self.initial_conv(x)
        
        # Reshape to add the sequence dimension
        # [batch_size, features, height, width] -> [batch_size, seq_length, features, height, width]
        x = x.unsqueeze(1).repeat(1, self.seq_length, 1, 1, 1)
        
        # Apply residual blocks with time distribution
        for res_block in self.res_blocks:
            x = res_block(x)
        
        # Apply spatial attention
        x = self.spatial_attention(x)
        
        # Global average pooling to reduce spatial dimensions
        x = x.view(batch_size * self.seq_length, -1, height, width)
        x = self.global_avg_pool(x)
        x = x.view(batch_size, self.seq_length, -1)
        
        # Apply LSTM for temporal processing
        if self.use_lstm:
            x, _ = self.lstm(x)
        
        # Apply temporal attention to focus on important time steps
        x = self.temporal_attention(x)
        
        # Reshape for decoder
        x = x.view(batch_size, -1, 1, 1)
        
        # Expand spatially to match original dimensions
        x = x.expand(-1, -1, height, width)
        
        # Final prediction
        x = self.decoder(x)
        
        return x

class ClimateTransformerCNN(nn.Module):
    """
    Transformer-based model for climate emulation
    Combines self-attention for temporal processing with CNN for spatial features
    """
    def __init__(
        self, 
        n_input_channels, 
        n_output_channels, 
        seq_length=12,
        kernel_size=5, 
        init_dim=64, 
        depth=4, 
        n_heads=8,
        dropout_rate=0.3
    ):
        super(ClimateTransformerCNN, self).__init__()
        
        self.seq_length = seq_length
        
        # Initial convolution applied to each time step
        self.initial_conv = nn.Sequential(
            nn.Conv2d(n_input_channels, init_dim, kernel_size=kernel_size, 
                     padding=kernel_size//2),
            nn.BatchNorm2d(init_dim),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        # Time-distributed CNN blocks
        self.res_blocks = nn.ModuleList()
        current_dim = init_dim
        
        for i in range(depth):
            out_dim = current_dim * 2 if i < depth - 1 else current_dim
            self.res_blocks.append(
                TimeDistributed(ResidualBlock(current_dim, out_dim))
            )
            if i < depth - 1:
                current_dim *= 2
        
        # Spatial attention
        self.spatial_attention = TimeDistributed(SpatialAttention(current_dim))
        
        # Global average pooling to reduce spatial dimensions
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        
        # Positional encoding for transformer
        self.pos_encoder = nn.Parameter(
            torch.zeros(1, 12, current_dim)
        )
        
        # Transformer encoder layer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=current_dim,
            nhead=n_heads,
            dim_feedforward=current_dim*4,
            dropout=dropout_rate,
            activation='gelu',
            batch_first=True
        )
        
        # Transformer encoder
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, 
            num_layers=2
        )
        
        # Final prediction layers
        self.decoder = nn.Sequential(
            nn.Conv2d(current_dim, current_dim, kernel_size=3, padding=1),
            nn.BatchNorm2d(current_dim),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(dropout_rate),
            nn.Conv2d(current_dim, current_dim//2, kernel_size=3, padding=1),
            nn.BatchNorm2d(current_dim//2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(current_dim//2, n_output_channels, kernel_size=1)
        )

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        
        # Instead of trying to reshape the input based on sequence length,
        # we'll adjust our approach to handle the actual input format
        
        # The input has 5 channels, which suggests it's a single time step with 5 features
        # Let's replicate this input across the sequence length
        
        # Remove the debug print in production code
        # print(f"Input shape: {x.shape}, Total elements: {x.numel()}")
        
        # Apply initial convolution directly
        x = self.initial_conv(x)
        
        # Reshape to add the sequence dimension
        # [batch_size, features, height, width] -> [batch_size, seq_length, features, height, width]
        x = x.unsqueeze(1).repeat(1, self.seq_length, 1, 1, 1)
        
        # Apply residual blocks with time distribution
        for res_block in self.res_blocks:
            x = res_block(x)
        
        # Apply spatial attention
        x = self.spatial_attention(x)
        
        # Global average pooling to reduce spatial dimensions
        x = x.view(batch_size * self.seq_length, -1, height, width)
        x = self.global_avg_pool(x)
        x = x.view(batch_size, self.seq_length, -1)
        
        # Add positional encoding
        if self.pos_encoder.size(1) != self.seq_length:
            self.pos_encoder = nn.Parameter(
                torch.zeros(1, self.seq_length, self.pos_encoder.size(2))
            )
        x = x + self.pos_encoder
        
        # Apply transformer encoder
        x = self.transformer_encoder(x)
        
        # Take the last time step's output
        x = x[:, -1]
        
        # Reshape for decoder
        x = x.view(batch_size, -1, 1, 1)
        
        # Expand spatially to match original dimensions
        x = x.expand(-1, -1, height, width)
        
        # Final prediction
        x = self.decoder(x)
        
        return x 

In [8]:
from pytorch_lightning.loggers import CSVLogger
import pandas as pd

csv_logger = CSVLogger(save_dir="logs/", name="my_experiment")


In [17]:
# from models import SequentialClimateCNN

datamodule = ClimateDataModule(**config["data"])
model = ClimateTransformerCNN(
    n_input_channels=len(config["data"]["input_vars"]),
    n_output_channels=len(config["data"]["output_vars"]),
    seq_length=config["data"].get("seq_length", 12),
    kernel_size=config["model"].get("kernel_size", 5),
    dropout_rate=config["model"].get("dropout_rate", 0.3),
    init_dim=32
)
lightning_module = ClimateEmulationModule(model, learning_rate=config["training"]["lr"])

trainer = pl.Trainer(**config["trainer"], logger = csv_logger)
trainer.fit(lightning_module, datamodule=datamodule) 

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/wcw003/.local/lib/python3.11/site-packages/zarr/core/group.py:3301: UserWarning: Object at .DS_Store is not recognized as a component of a Zarr hierarchy.
  warnings.warn(


Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                  | Params | Mode 
------------------------------------------------------------
0 | model     | ClimateTransformerCNN | 9.0 M  | train
1 | criterion | MSELoss               | 0      | train
------------------------------------------------------------
9.0 M     Trainable params
0         Non-trainable params
9.0 M     Total params
36.086    Total estimated model params size (MB)
80        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

[VAL] tas: RMSE=15.5908, Time-Mean RMSE=14.8272, Time-Stddev MAE=2.6516
[VAL] pr: RMSE=3.5110, Time-Mean RMSE=2.1750, Time-Stddev MAE=2.0854


Validation: |          | 0/? [00:00<?, ?it/s]

[VAL] tas: RMSE=15.8884, Time-Mean RMSE=15.1008, Time-Stddev MAE=3.1245
[VAL] pr: RMSE=3.5203, Time-Mean RMSE=2.1896, Time-Stddev MAE=2.1073


`Trainer.fit` stopped: `max_epochs=15` reached.


In [24]:
df = pd.read_csv(f"{csv_logger.log_dir}/metrics.csv")
df

,epoch,step,train/loss,val/loss,val/pr/rmse,val/pr/time_mean_rmse,val/pr/time_std_mae,val/tas/rmse,val/tas/time_mean_rmse,val/tas/time_std_mae
0,0,49,0.906758,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,49,0.864546,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,99,0.886342,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,149,0.812010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,199,0.864929,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
112,14,4899,0.834978,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,14,4949,0.840607,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,14,4999,0.845257,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,14,5049,0.817505,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
csv_logger = CSVLogger(save_dir="logs/", name="baseline_10")

In [28]:
#original
datamodule = ClimateDataModule(**config["data"])
model = SimpleCNN(
    n_input_channels=len(config["data"]["input_vars"]),
    n_output_channels=len(config["data"]["output_vars"]),
    **{k: v for k, v in config["model"].items() if k != "type"}
)
lightning_module = ClimateEmulationModule(model, learning_rate=config["training"]["lr"])

trainer = pl.Trainer(**config["trainer"], logger = csv_logger)
trainer.fit(lightning_module, datamodule=datamodule)   # Training

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/wcw003/.local/lib/python3.11/site-packages/zarr/core/group.py:3301: UserWarning: Object at .DS_Store is not recognized as a component of a Zarr hierarchy.
  warnings.warn(


Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | model     | SimpleCNN | 27.4 M | train
1 | criterion | MSELoss   | 0      | train
------------------------------------------------
27.4 M    Trainable params
0         Non-trainable params
27.4 M    Total params
109.507   Total estimated model params size (MB)
47        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

[VAL] tas: RMSE=2.2043, Time-Mean RMSE=1.0390, Time-Stddev MAE=0.5223
[VAL] pr: RMSE=2.0395, Time-Mean RMSE=0.3944, Time-Stddev MAE=0.8404


Validation: |          | 0/? [00:00<?, ?it/s]

[VAL] tas: RMSE=2.0285, Time-Mean RMSE=1.0015, Time-Stddev MAE=0.5026
[VAL] pr: RMSE=2.0292, Time-Mean RMSE=0.3596, Time-Stddev MAE=0.8042


`Trainer.fit` stopped: `max_epochs=15` reached.


# Test model

**IMPORTANT:** Please note that the test metrics will be bad because the test targets have been corrupted on the public Kaggle dataset.
The purpose of testing below is to generate the Kaggle submission file based on your model's predictions, which you can submit to the competition.

In [29]:
trainer.test(lightning_module, datamodule=datamodule) 

/home/wcw003/.local/lib/python3.11/site-packages/zarr/core/group.py:3301: UserWarning: Object at .DS_Store is not recognized as a component of a Zarr hierarchy.
  warnings.warn(


Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[TEST] tas: RMSE=289.9964, Time-Mean RMSE=289.9575, Time-Stddev MAE=3.3582
[TEST] pr: RMSE=4.1643, Time-Mean RMSE=3.7216, Time-Stddev MAE=1.3198
✅ Submission saved to: submissions/kaggle_submission_20250516_001026.csv


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/pr/rmse        │     4.164330959320068     │
│  test/pr/time_mean_rmse   │     3.721630573272705     │
│   test/pr/time_std_mae    │     1.319814682006836     │
│       test/tas/rmse       │     289.9964599609375     │
│  test/tas/time_mean_rmse  │     289.9574890136719     │
│   test/tas/time_std_mae   │    3.3581554889678955     │
└───────────────────────────┴───────────────────────────┘

[{'test/tas/rmse': 289.9964599609375,
  'test/tas/time_mean_rmse': 289.9574890136719,
  'test/tas/time_std_mae': 3.3581554889678955,
  'test/pr/rmse': 4.164330959320068,
  'test/pr/time_mean_rmse': 3.721630573272705,
  'test/pr/time_std_mae': 1.319814682006836}]

In [ ]:
import pandas as pd
df = pd.read_csv(f"{csv_logger.log_dir}/metrics.csv")
df

In [11]:
df = pd.read_csv('logs/baseline_10/version_0/metrics.csv')

In [13]:
# baseline for 15 epochs
df[df['val/tas/rmse'].notna()]

,epoch,step,test/pr/rmse,test/pr/time_mean_rmse,test/pr/time_std_mae,test/tas/rmse,test/tas/time_mean_rmse,test/tas/time_std_mae,train/loss,val/loss,val/pr/rmse,val/pr/time_mean_rmse,val/pr/time_std_mae,val/tas/rmse,val/tas/time_mean_rmse,val/tas/time_std_mae
6,0,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.413533,3.044483,1.353611,1.544218,5.595884,3.945855,1.363540
14,1,675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.352440,2.829718,1.033430,1.514174,4.943322,3.621779,1.284734
22,2,1013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.310559,2.671203,0.729546,1.448786,4.128765,2.749595,1.202235
30,3,1351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.312051,2.668728,0.644865,1.476117,4.330984,2.389239,1.126163
37,4,1689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.286025,2.607461,0.586888,1.414261,3.498919,1.860331,1.249455
45,5,2027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.290555,2.645019,0.760569,1.328599,3.585017,2.110446,0.997285
53,6,2365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.266779,2.498491,0.541491,1.179986,3.680653,2.236222,0.809780
61,7,2703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.217828,2.268676,0.558791,1.040210,2.921343,1.419113,0.631662
68,8,3041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.197965,2.163139,0.459045,0.973214,2.727245,1.228776,0.837583
76,9,3379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.189973,2.120011,0.434374,0.927952,2.728606,1.448274,0.648062


In [14]:
df_1 = pd.read_csv('logs/ClimateTransformerCNN_15/version_0/metrics.csv')

In [16]:
# ClimateTransformerCNN for 15 epochs
df_1[df_1['val/tas/rmse'].notna()]

,epoch,step,test/pr/rmse,test/pr/time_mean_rmse,test/pr/time_std_mae,test/tas/rmse,test/tas/time_mean_rmse,test/tas/time_std_mae,train/loss,val/loss,val/pr/rmse,val/pr/time_mean_rmse,val/pr/time_std_mae,val/tas/rmse,val/tas/time_mean_rmse,val/tas/time_std_mae
7,0,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.860833,3.511637,2.175080,2.104146,15.221120,14.374287,2.655231
15,1,675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.861554,3.504153,2.163297,2.136274,15.250577,14.420325,3.506759
23,2,1013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.854624,3.510213,2.171349,2.034225,15.438791,14.653691,2.405034
31,3,1351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.855240,3.514596,2.180201,2.099018,15.519410,14.693075,2.278483
38,4,1689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.852906,3.514411,2.180029,2.119374,15.464151,14.686107,2.390196
46,5,2027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.853600,3.508709,2.171291,2.099084,15.419410,14.622574,2.289880
54,6,2365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.853228,3.509523,2.172109,2.111347,15.211109,14.409259,2.291581
62,7,2703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851899,3.519500,2.187137,2.045025,15.855614,15.096375,2.299298
69,8,3041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.850602,3.513256,2.178537,2.087612,15.305568,14.522534,2.340683
77,9,3379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851674,3.519984,2.189481,2.100095,15.615163,14.826754,2.620029


### Plotting Utils


In [ ]:
def plot_comparison(true_xr, pred_xr, title, cmap='viridis', diff_cmap='RdBu_r', metric=None):
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))

    vmin = min(true_xr.min().item(), pred_xr.min().item())
    vmax = max(true_xr.max().item(), pred_xr.max().item())

    # Ground truth
    true_xr.plot(ax=axs[0], cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=True)
    axs[0].set_title(f"{title} (Ground Truth)")

    # Prediction
    pred_xr.plot(ax=axs[1], cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=True)
    axs[1].set_title(f"{title} (Prediction)")

    # Difference
    diff = pred_xr - true_xr
    abs_max = np.max(np.abs(diff))
    diff.plot(ax=axs[2], cmap=diff_cmap, vmin=-abs_max, vmax=abs_max, add_colorbar=True)
    axs[2].set_title(f"{title} (Difference) {f'- {metric:.4f}' if metric else ''}")

    plt.tight_layout()
    plt.show()


### 🖼️ Visualizing Validation Predictions

This cell loads saved validation predictions and compares them to the ground truth using spatial plots. These visualizations help you qualitatively assess your model's performance.

For each output variable (`tas`, `pr`), we visualize:

- **📈 Time-Mean Map**: The 10-year average spatial pattern for both prediction and ground truth. Helps identify long-term biases or spatial shifts.
- **📊 Time-Stddev Map**: Shows the standard deviation across time for each grid cell — useful for assessing how well the model captures **temporal variability** at each location.
- **🕓 Random Timestep Sample**: Visual comparison of prediction vs ground truth for a single month. Useful for spotting fine-grained anomalies or errors in specific months.

> These plots provide intuition beyond metrics and are useful for debugging spatial or temporal model failures.


In [ ]:
# Load validation predictions
# make sure to have run the validation loop at least once
val_preds = np.load("val_preds.npy")
val_trues = np.load("val_trues.npy")

lat, lon = datamodule.get_coords()
output_vars = datamodule.output_vars
time = np.arange(val_preds.shape[0])

for i, var in enumerate(output_vars):
    pred_xr = xr.DataArray(val_preds[:, i], dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})
    true_xr = xr.DataArray(val_trues[:, i], dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})

    # --- Time Mean ---
    plot_comparison(true_xr.mean("time"), pred_xr.mean("time"), f"{var} Val Time-Mean")

    # --- Time Stddev ---
    plot_comparison(true_xr.std("time"), pred_xr.std("time"), f"{var} Val Time-Stddev", cmap="plasma")

    # --- Random timestep ---
    t_idx = np.random.randint(0, len(time))
    plot_comparison(true_xr.isel(time=t_idx), pred_xr.isel(time=t_idx), f"{var} Val Sample Timestep {t_idx}")


## 🧪 Final Notes

This notebook is meant to serve as a **baseline template** — a starting point to help you get up and running quickly with the climate emulation challenge.

You are **not** required to stick to this exact setup. In fact, we **encourage** you to:

- 🔁 Build on top of the provided `DataModule`. 
- 🧠 Use your own model architectures or training pipelines that you’re more comfortable with 
- ⚗️ Experiment with ideas  
- 🥇 Compete creatively to climb the Kaggle leaderboard  
- 🙌 Most importantly: **have fun** and **learn as much as you can** along the way

This challenge simulates a real-world scientific problem, and there’s no single "correct" approach — so be curious, experiment boldly, and make it your own!
